In [1]:
print(spark.version)


VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
2,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.4.1-amzn-2

In [68]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, LongType, TimestampNTZType
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pprint import pprint
from pyspark.sql.functions import col, unix_timestamp, lit, hour, dayofweek, udf, when, broadcast, count, avg, sum, when, max, min
from typing import Optional, Any
from pyspark.sql import DataFrame
import boto3
import tempfile
import os
import uuid
import shutil
from enum import Enum
from pathlib import Path
from datetime import datetime, timezone

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# TASK 2 Імпорт, уніфікація та об’єднання

In [106]:
S3_BUCKET_NAME = "robot-dreams-source-data"
YELLOW_TAXI_PATH = f"s3://{S3_BUCKET_NAME}/home-work-1/nyc_taxi/yellow/"
GREEN_TAXI_PATH = f"s3://{S3_BUCKET_NAME}/home-work-1/nyc_taxi/green/"
ZONE_LOOKUP_PATH = f"s3://{S3_BUCKET_NAME}/Lecture_3/nyc_taxi/taxi_zone_lookup.csv"

UNIFIED_YELLOW_TAXI_PATH = f"s3://{S3_BUCKET_NAME}/home-work-1-unified/nyc_taxi/yellow/"
UNIFIED_GREEN_TAXI_PATH = f"s3://{S3_BUCKET_NAME}/home-work-1-unified/nyc_taxi/green/"

print(f"Using Unified Yellow Taxi data from: {UNIFIED_YELLOW_TAXI_PATH}")
print(f"Using Unified Green Taxi data from: {YELLOW_TAXI_PATH}")
print(f"Using Yellow Taxi data from: {YELLOW_TAXI_PATH}")
print(f"Using Green Taxi data from: {GREEN_TAXI_PATH}")
print(f"Using Zone Lookup data from: {ZONE_LOOKUP_PATH}")
spark.conf.set("spark.sql.parquet.mergeSchema", "false")

    
S3_TEMP_BUCKET = "zhogolev-pv-temp-files"

YELLOW_TEMP_FOLDER = f"s3://{S3_TEMP_BUCKET}/temp/yellow_taxi_combined_da284725/"
GREEN_TEMP_FOLDER = f"s3://{S3_TEMP_BUCKET}/temp/green_taxi_combined_da284725/"

print(f"Yellow taxi source path: {YELLOW_TAXI_PATH}")
print(f"Green taxi source path: {GREEN_TAXI_PATH}")
print(f"S3 temp bucket: {S3_TEMP_BUCKET}")
print(f"Yellow taxi S3 temp folder: {YELLOW_TEMP_FOLDER}")
print(f"Green taxi S3 temp folder: {GREEN_TEMP_FOLDER}")

temp_folders = []
temp_folders.extend([YELLOW_TEMP_FOLDER, GREEN_TEMP_FOLDER])

#CURRENT_DAY
current_date = datetime.now(timezone.utc).date().isoformat()

class TaxiTypeEnum(Enum):
    GREEN = "GREEN"
    YELLOW = "YELLOW"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Using Unified Yellow Taxi data from: s3://robot-dreams-source-data/home-work-1-unified/nyc_taxi/yellow/
Using Unified Green Taxi data from: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/
Using Yellow Taxi data from: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/
Using Green Taxi data from: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/
Using Zone Lookup data from: s3://robot-dreams-source-data/Lecture_3/nyc_taxi/taxi_zone_lookup.csv
Yellow taxi source path: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/
Green taxi source path: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/
S3 temp bucket: zhogolev-pv-temp-files
Yellow taxi S3 temp folder: s3://zhogolev-pv-temp-files/temp/yellow_taxi_combined_da284725/
Green taxi S3 temp folder: s3://zhogolev-pv-temp-files/temp/green_taxi_combined_da284725/

In [13]:
def show_schema(df):
    for col in df.schema:
        print(col)
        
        
def inspect_parquet_schema(path: Optional[str] = None, df: Any = None, is_read: bool = True):
    """
    Inspect the actual schema of parquet files without trying to enforce a schema
    """
    try:
        if is_read:
            df = spark.read.option("recursiveFileLookup", "true").parquet(path)
        print("Actual schema in the files:")
        schema = df.schema
        for col in schema:
            print(col)
        print("\n")
        
    except Exception as e:
        print(f"Error inspecting schema: {e}")
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### BLOCK PROCESSING TIPES DATASETS

In [5]:

def list_files(bucket_name, s3_path):
    s3_client = boto3.client('s3')

    prefix = s3_path.replace(f"s3://{bucket_name}/", "")
    
    paginator = s3_client.get_paginator('list_objects_v2')
    parquet_files = []
    
    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
        if 'Contents' in page:
            for obj in page['Contents']:
                if obj['Key'].endswith('.parquet'):
                    full_path = f"s3://{bucket_name}/{obj['Key']}"
                    parquet_files.append(full_path)
    
    return sorted(parquet_files)

yellow_files = list_files(S3_BUCKET_NAME, YELLOW_TAXI_PATH)
green_files = list_files(S3_BUCKET_NAME, GREEN_TAXI_PATH)

print("YELLOW TAXI PARQUET FILES:")
print("=" * 100)
for file in yellow_files:
    print(file)

print(f"\nGREEN TAXI PARQUET FILES:")
print("=" * 100)
for file in green_files:
    print(file)

print(f"\nFile counts:")
print(f"Yellow: {len(yellow_files)}")
print(f"Green: {len(green_files)}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

YELLOW TAXI PARQUET FILES:
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-01.parquet
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-02.parquet
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-03.parquet
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-04.parquet
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-05.parquet
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-06.parquet
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-07.parquet
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-08.parquet
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-09.parquet
s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-10.parquet
s3://robot-dreams-sourc

In [6]:
#Inspect yellow taxi dataset schema
inspect_parquet_schema(YELLOW_TAXI_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Actual schema in the files:
StructField('VendorID', LongType(), True)
StructField('tpep_pickup_datetime', TimestampNTZType(), True)
StructField('tpep_dropoff_datetime', TimestampNTZType(), True)
StructField('passenger_count', LongType(), True)
StructField('trip_distance', DoubleType(), True)
StructField('RatecodeID', LongType(), True)
StructField('store_and_fwd_flag', StringType(), True)
StructField('PULocationID', LongType(), True)
StructField('DOLocationID', LongType(), True)
StructField('payment_type', LongType(), True)
StructField('fare_amount', DoubleType(), True)
StructField('extra', DoubleType(), True)
StructField('mta_tax', DoubleType(), True)
StructField('tip_amount', DoubleType(), True)
StructField('tolls_amount', DoubleType(), True)
StructField('improvement_surcharge', DoubleType(), True)
StructField('total_amount', DoubleType(), True)
StructField('congestion_surcharge', DoubleType(), True)
StructField('airport_fee', IntegerType(), True)

In [8]:
#Inspect green taxi dataset schema
inspect_parquet_schema(GREEN_TAXI_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Actual schema in the files:
StructField('VendorID', LongType(), True)
StructField('lpep_pickup_datetime', TimestampNTZType(), True)
StructField('lpep_dropoff_datetime', TimestampNTZType(), True)
StructField('store_and_fwd_flag', StringType(), True)
StructField('RatecodeID', LongType(), True)
StructField('PULocationID', LongType(), True)
StructField('DOLocationID', LongType(), True)
StructField('passenger_count', LongType(), True)
StructField('trip_distance', DoubleType(), True)
StructField('fare_amount', DoubleType(), True)
StructField('extra', DoubleType(), True)
StructField('mta_tax', DoubleType(), True)
StructField('tip_amount', DoubleType(), True)
StructField('tolls_amount', DoubleType(), True)
StructField('ehail_fee', IntegerType(), True)
StructField('improvement_surcharge', DoubleType(), True)
StructField('total_amount', DoubleType(), True)
StructField('payment_type', LongType(), True)
StructField('trip_type', DoubleType(), True)
StructField('congestion_surcharge', IntegerType(),

In [73]:
# Schemas for read base source files
yellow_taxi_schema_universal = StructType([
    StructField("VendorID", LongType(), True),
    StructField("tpep_pickup_datetime", TimestampNTZType(), True),
    StructField("tpep_dropoff_datetime", TimestampNTZType(), True),
    StructField("passenger_count", DoubleType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("RatecodeID", DoubleType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("PULocationID", DoubleType(), True),
    StructField("DOLocationID", DoubleType(), True),
    StructField("payment_type", DoubleType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("congestion_surcharge", DoubleType(), True),
    StructField("airport_fee", DoubleType(), True)
])

green_taxi_schema_universal = StructType([
    StructField('VendorID', LongType(), True),
    StructField('lpep_pickup_datetime', TimestampNTZType(), True),
    StructField('lpep_dropoff_datetime', TimestampNTZType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('RatecodeID', DoubleType(), True),
    StructField('PULocationID', LongType(), True),
    StructField('DOLocationID', LongType(), True),
    StructField('passenger_count', LongType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('ehail_fee', IntegerType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('payment_type', LongType(), True),
    StructField('trip_type', DoubleType(), True),
    StructField('congestion_surcharge', IntegerType(), True)
])

#Schema read if happend some unexpected problems
yellow_taxi_schema_flexible = StructType([
    StructField("VendorID", StringType(), True),
    StructField("tpep_pickup_datetime", TimestampNTZType(), True),
    StructField("tpep_dropoff_datetime", TimestampNTZType(), True),
    StructField("passenger_count", StringType(), True),
    StructField("trip_distance", StringType(), True),
    StructField("RatecodeID", StringType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("PULocationID", StringType(), True),
    StructField("DOLocationID", StringType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", StringType(), True),
    StructField("extra", StringType(), True),
    StructField("mta_tax", StringType(), True),
    StructField("tip_amount", StringType(), True),
    StructField("tolls_amount", StringType(), True),
    StructField("improvement_surcharge", StringType(), True),
    StructField("total_amount", StringType(), True),
    StructField("congestion_surcharge", StringType(), True),
    StructField("airport_fee", StringType(), True)
])

green_taxi_schema_flexible = StructType([
    StructField("VendorID", StringType(), True),
    StructField("lpep_pickup_datetime", TimestampNTZType(), True),
    StructField("lpep_dropoff_datetime", TimestampNTZType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("RatecodeID", StringType(), True),
    StructField("PULocationID", StringType(), True),
    StructField("DOLocationID", StringType(), True),
    StructField("passenger_count", StringType(), True),
    StructField("trip_distance", StringType(), True),
    StructField("fare_amount", StringType(), True),
    StructField("extra", StringType(), True),
    StructField("mta_tax", StringType(), True),
    StructField("tip_amount", StringType(), True),
    StructField("tolls_amount", StringType(), True),
    StructField("ehail_fee", StringType(), True),
    StructField("improvement_surcharge", StringType(), True),
    StructField("total_amount", StringType(), True),
    StructField("payment_type", StringType(), True),
    StructField("trip_type", StringType(), True),
    StructField("congestion_surcharge", StringType(), True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
# Func read singl parquet file
def read_single_parquet_file(file_path: str, taxi_type: TaxiTypeEnum) -> DataFrame:
    """Read a single parquet file with schema enforcement"""
    try:
        df = spark.read \
            .option("mergeSchema", "true") \
            .option("inferSchema", "false") \
            .parquet(file_path)
        
        print(f" Read {os.path.basename(file_path)} without explicit schema")
        return df

    except Exception as e:
        print(f"Failed without schema: {str(e)[:80]}...")

        try:
            if taxi_type == TaxiTypeEnum.YELLOW:
                target_schema = yellow_taxi_schema_universal
            else:
                target_schema = green_taxi_schema_universal

            df = spark.read \
                .option("mergeSchema", "false") \
                .option("inferSchema", "false") \
                .schema(target_schema) \
                .parquet(file_path)

            print(f"Read {os.path.basename(file_path)} with universal schema")
            return df

        except Exception as e2:
            print(f"Failed with universal schema: {str(e2)[:80]}...")

            if taxi_type == TaxiTypeEnum.YELLOW:
                target_schema = yellow_taxi_schema_flexible
            else:
                target_schema = green_taxi_schema_flexible

            df = spark.read \
                .option("mergeSchema", "false") \
                .option("inferSchema", "false") \
                .schema(target_schema) \
                .parquet(file_path)

            print(f"Read {os.path.basename(file_path)} with flexible schema")
            return df

print("Single file reading function defined")

#Func cast types
def apply_type_casting(df: DataFrame) -> DataFrame:
    """Apply type casting to DataFrame"""
    
    casting_rules = {
        "VendorID": "long",
        "passenger_count": "long", 
        "trip_distance": "double",
        "RatecodeID": "double",
        "store_and_fwd_flag": "string",
        "PULocationID": "long",
        "DOLocationID": "long",
        "payment_type": "long",
        "fare_amount": "double",
        "extra": "double",
        "mta_tax": "double",
        "tip_amount": "double",
        "tolls_amount": "double",
        "improvement_surcharge": "double",
        "total_amount": "double",
        "congestion_surcharge": "double",
        "airport_fee": "double",
        "ehail_fee": "double",
        "trip_type": "long"
    }
    
    cast_columns = []
    for column, cast_type in casting_rules.items():
        if column in df.columns:
            df = df.withColumn(column, col(column).cast(cast_type))
            cast_columns.append(column)
    
    print(f"Applied casting to {len(cast_columns)} columns")
    return df

print("Type casting function defined")

temp_files = []

print("Save to folder function defined")

# Func clear processed files bucket.
def clear_s3_temp_folders():
    """Clean up S3 temporary folders using Spark"""
    import boto3
    
    try:
        s3_client = boto3.client('s3')
        bucket_name = S3_TEMP_BUCKET
        
        for s3_folder_path in temp_folders:
            try:
                s3_key_prefix = s3_folder_path.replace(f"s3://{bucket_name}/", "")
                
                print(f"Cleaning up S3 path: {s3_folder_path}")
                response = s3_client.list_objects_v2(
                    Bucket=bucket_name,
                    Prefix=s3_key_prefix
                )
                
                if 'Contents' in response:
                    objects_to_delete = [{'Key': obj['Key']} for obj in response['Contents']]
                    
                    if objects_to_delete:
                        s3_client.delete_objects(
                            Bucket=bucket_name,
                            Delete={'Objects': objects_to_delete}
                        )
                        print(f"✓ Deleted {len(objects_to_delete)} objects from {s3_folder_path}")
                    else:
                        print(f"✓ No objects found in {s3_folder_path}")
                else:
                    print(f"✓ Folder {s3_folder_path} is already empty or doesn't exist")
                    
            except Exception as e:
                print(f"⚠ Error cleaning up {s3_folder_path}: {e}")
                
        temp_folders.clear()
        print("S3 cleanup completed!")
        
    except Exception as e:
        print(f"✗ Error during S3 cleanup: {e}")
        print("You may need to clean up manually using AWS console or CLI")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Single file reading function defined
Type casting function defined
Save to folder function defined

In [58]:
#Processing base source file one by one
def process_files_one_by_one_to_s3(file_paths: list, taxi_type: TaxiTypeEnum, target_s3_folder: str) -> dict:
    """Process files one by one and save each to S3 directly in the main folder"""
    
    total_files = len(file_paths)
    successful_files = 0
    failed_files = []
    processed_files_info = []
    
    print(f"Processing {total_files} {taxi_type.value} files one by one to S3...")
    print(f"Target S3 folder: {target_s3_folder}")
    
    for i, file_path in enumerate(file_paths, 1):
        try:
            print(f"[{i}/{total_files}] Processing: {os.path.basename(file_path)}")

            df = read_single_parquet_file(file_path, taxi_type)
 
            df = apply_type_casting(df)

            record_count = df.count()
        
            source_filename = os.path.basename(file_path).replace('.parquet', '').replace('_', '-')
            subfolder_name = f"{taxi_type.value.lower()}-{i:04d}-{source_filename}"
            subfolder_path = f"{target_s3_folder}{subfolder_name}/"
            
            df.coalesce(1) \
                .write \
                .mode("overwrite") \
                .option("compression", "snappy") \
                .parquet(subfolder_path)
            
            show_schema(df)
            
            successful_files += 1
            processed_files_info.append({
                'file_number': i,
                'source_file': os.path.basename(file_path),
                'record_count': record_count
            })
            
            print(f"Processed {record_count:,} records")

            del df
            
        except Exception as e:
            print(f"Failed: {str(e)[:150]}...")
            failed_files.append({
                'file_path': file_path,
                'error': str(e)
            })
            continue
    
    total_records = sum(info['record_count'] for info in processed_files_info)
    
    print(f"\n{taxi_type.value} PROCESSING SUMMARY:")
    print(f" Successfully processed: {successful_files}/{total_files} files")
    print(f" Total records processed: {total_records:,}")
    print(f" S3 folder: {target_s3_folder}")
    print(f" Failed files: {len(failed_files)}")
    
    if failed_files:
        print(f"\n Failed files:")
        for failed in failed_files[:5]:  # Show first 5 failures
            print(f"    - {os.path.basename(failed['file_path'])}: {failed['error'][:100]}...")
    
    return {
        'taxi_type': taxi_type.value,
        'successful_files': successful_files,
        'total_files': total_files,
        'total_records': total_records,
        'failed_files': failed_files,
        'processed_files_info': processed_files_info,
        's3_folder': target_s3_folder
    }

print("File-by-file processing function defined")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

File-by-file processing function defined

In [55]:
# FOR TEST ONLY. One file processing
test_read = []
file_path_test = yellow_files[0]
test_read.append(file_path_test)
print(test_read)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-01.parquet']

In [59]:
#Processing yellow taxi dataset
yellow_results = None

if yellow_files:
    print("=" * 70)
    print("PROCESSING YELLOW TAXI DATA - FILE BY FILE TO S3")
    print("=" * 70)
    
    
    try:
        yellow_results = process_files_one_by_one_to_s3(
            yellow_files, 
            TaxiTypeEnum.YELLOW, 
            YELLOW_TEMP_FOLDER
        )
        
        print(f"YELLOW TAXI COMPLETED:")
        print(f"Files processed: {yellow_results['successful_files']}/{yellow_results['total_files']}")
        print(f"Total records: {yellow_results['total_records']:,}")
        print(f"S3 location: {yellow_results['s3_folder']}")
        
    except Exception as e:
        print(f"✗ Error in yellow taxi processing: {e}")
        yellow_results = {'error': str(e)}
else:
    print("No yellow taxi files to process")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PROCESSING YELLOW TAXI DATA - FILE BY FILE TO S3
🚀 Processing 92 YELLOW files one by one to S3...
📁 Target S3 folder: s3://zhogolev-pv-temp-files/temp/yellow_taxi_combined_da284725/

[1/92] Processing: yellow_tripdata_2014-01.parquet
    ✓ Read yellow_tripdata_2014-01.parquet without explicit schema
    ✓ Applied casting to 16 columns
StructField('VendorID', LongType(), True)
StructField('tpep_pickup_datetime', TimestampNTZType(), True)
StructField('tpep_dropoff_datetime', TimestampNTZType(), True)
StructField('passenger_count', LongType(), True)
StructField('trip_distance', DoubleType(), True)
StructField('RatecodeID', DoubleType(), True)
StructField('store_and_fwd_flag', StringType(), True)
StructField('PULocationID', LongType(), True)
StructField('DOLocationID', LongType(), True)
StructField('payment_type', LongType(), True)
StructField('fare_amount', DoubleType(), True)
StructField('extra', DoubleType(), True)
StructField('mta_tax', DoubleType(), True)
StructField('tip_amount', Dou

In [75]:
#Processing green taxi dataset
green_results = None

if green_files:
    print("=" * 70)
    print("PROCESSING GREEN TAXI DATA - FILE BY FILE TO S3")
    print("=" * 70)
    
    try:
        green_results = process_files_one_by_one_to_s3(
            green_files, 
            TaxiTypeEnum.GREEN, 
            GREEN_TEMP_FOLDER
        )
        
        print(f"GREEN TAXI COMPLETED:")
        print(f"Files processed: {green_results['successful_files']}/{green_results['total_files']}")
        print(f"Total records: {green_results['total_records']:,}")
        print(f"S3 location: {green_results['s3_folder']}")
        
    except Exception as e:
        print(f"✗ Error in green taxi processing: {e}")
        green_results = {'error': str(e)}
else:
    print("No green taxi files to process")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-75:
Traceback (most recent call last):
  File "/usr/lib64/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hadoop/spark-magic/spark_monitoring_widget/src/awseditorssparkmonitoringwidget/cellmonitor.py", line 180, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1296



PROCESSING GREEN TAXI DATA - FILE BY FILE TO S3
🚀 Processing 117 GREEN files one by one to S3...
📁 Target S3 folder: s3://zhogolev-pv-temp-files/temp/green_taxi_combined_da284725/

[1/117] Processing: green_tripdata_2014-01.parquet
    ✓ Read green_tripdata_2014-01.parquet without explicit schema
    ✓ Applied casting to 18 columns
StructField('VendorID', LongType(), True)
StructField('lpep_pickup_datetime', TimestampNTZType(), True)
StructField('lpep_dropoff_datetime', TimestampNTZType(), True)
StructField('store_and_fwd_flag', StringType(), True)
StructField('RatecodeID', DoubleType(), True)
StructField('PULocationID', LongType(), True)
StructField('DOLocationID', LongType(), True)
StructField('passenger_count', LongType(), True)
StructField('trip_distance', DoubleType(), True)
StructField('fare_amount', DoubleType(), True)
StructField('extra', DoubleType(), True)
StructField('mta_tax', DoubleType(), True)
StructField('tip_amount', DoubleType(), True)
StructField('tolls_amount', Doub

### BLOCK WORK WITH TYPES PROCESSED DATA

In [109]:
print("1. Set up schema for read processed data...")
green_processed_schema = StructType([
    StructField('VendorID', LongType(), True),
    StructField('lpep_pickup_datetime', TimestampNTZType(), True),
    StructField('lpep_dropoff_datetime', TimestampNTZType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('RatecodeID', DoubleType(), True),
    StructField('PULocationID', LongType(), True),
    StructField('DOLocationID', LongType(), True),
    StructField('passenger_count', LongType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('ehail_fee', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('payment_type', LongType(), True),
    StructField('trip_type', LongType(), True),
    StructField('congestion_surcharge', DoubleType(), True)
])

yellow_processed_schema = StructType([
    StructField("VendorID", LongType(), True),
    StructField("tpep_pickup_datetime", TimestampNTZType(), True),
    StructField("tpep_dropoff_datetime", TimestampNTZType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("RatecodeID", DoubleType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("congestion_surcharge", DoubleType(), True),
    StructField("airport_fee", DoubleType(), True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1. Set up schema for read processed data...

In [110]:
print("2. Read processed_data...")
def verify_processed_data_safely(s3_folder: str, taxi_type: TaxiTypeEnum, schema) -> dict:
    """Safely verify processed data using sampling"""
    try:
        print(f"Verifying {taxi_type} processed data...")
        print(f"S3 folder: {s3_folder}")

        df = spark.read.schema(schema).option("recursiveFileLookup", "true").parquet(s3_folder)

        schema_info = {
            "columns": len(df.columns),
            "column_names": df.columns
        }

        sample_fraction = 0.005  
        sample_df = df.sample(withReplacement=False, fraction=sample_fraction, seed=42)
        sample_count = sample_df.count()
        estimated_total = int(sample_count / sample_fraction) if sample_count > 0 else 0

        print(f"Columns: {schema_info['columns']}")
        print(f"Estimated total records: ~{estimated_total:,}")
        print(f"Sample data (from {sample_fraction*100}% sample):")
        sample_df.limit(3).show(truncate=False)
        
        return df
        
    except Exception as e:
        print(f"    ✗ Verification failed: {e}")
        return {"error": str(e)}

# Read Yellow Taxi Data
yellow_df = verify_processed_data_safely(YELLOW_TEMP_FOLDER, TaxiTypeEnum.YELLOW, yellow_processed_schema)

# Read Green Taxi Data  
green_df = verify_processed_data_safely(GREEN_TEMP_FOLDER, TaxiTypeEnum.GREEN, green_processed_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2. Read processed_data...
Verifying TaxiTypeEnum.YELLOW processed data...
S3 folder: s3://zhogolev-pv-temp-files/temp/yellow_taxi_combined_da284725/
Columns: 19
Estimated total records: ~744,854,800
Sample data (from 0.5% sample):
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+---

In [55]:
print("3. Analyzing schemas...")

print("Yellow taxi columns:")
yellow_schema = yellow_df.schema
show_schema(yellow_df)
set_yellow_schema = set(yellow_schema)

print("-"*100)

print("Green taxi columns:")
green_schema = green_df.schema
show_schema(green_df)
set_green_schema = set(green_schema)

print("-"*100)

# get diff between columes
diff_green_yellow = list(set_green_schema - set_yellow_schema)
print(f"Diff green_yellow {diff_green_yellow}")

diff_yellow_green = list(set_yellow_schema - set_green_schema)
print(f"Diff yellow_green {diff_yellow_green}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3. Analyzing schemas...
Yellow taxi columns:
StructField('VendorID', LongType(), True)
StructField('tpep_pickup_datetime', TimestampNTZType(), True)
StructField('tpep_dropoff_datetime', TimestampNTZType(), True)
StructField('passenger_count', LongType(), True)
StructField('trip_distance', DoubleType(), True)
StructField('RatecodeID', DoubleType(), True)
StructField('store_and_fwd_flag', StringType(), True)
StructField('PULocationID', LongType(), True)
StructField('DOLocationID', LongType(), True)
StructField('payment_type', LongType(), True)
StructField('fare_amount', DoubleType(), True)
StructField('extra', DoubleType(), True)
StructField('mta_tax', DoubleType(), True)
StructField('tip_amount', DoubleType(), True)
StructField('tolls_amount', DoubleType(), True)
StructField('improvement_surcharge', DoubleType(), True)
StructField('total_amount', DoubleType(), True)
StructField('congestion_surcharge', DoubleType(), True)
StructField('airport_fee', DoubleType(), True)
-------------------

In [56]:
print("4. Performing union...")

yellow_df_standardized = yellow_df.select(
    col("VendorID"),
    col("tpep_pickup_datetime").alias("pickup_datetime"),
    col("tpep_dropoff_datetime").alias("dropoff_datetime"),
    col("passenger_count"),
    col("trip_distance"),
    col("RatecodeID"),
    col("store_and_fwd_flag"),
    col("PULocationID"),
    col("DOLocationID"),
    col("payment_type"),
    col("fare_amount"),
    col("extra"),
    col("mta_tax"),
    col("tip_amount"),
    col("tolls_amount"),
    lit(None).cast("double").alias("ehail_fee"), 
    col("improvement_surcharge"),
    col("total_amount"),
    lit(None).cast("long").alias("trip_type"),
    col("congestion_surcharge"),
    col("airport_fee"),
    lit("yellow").alias("taxi_type")
)

green_df_standardized = green_df.select(
    col("VendorID"),
    col("lpep_pickup_datetime").alias("pickup_datetime"), 
    col("lpep_dropoff_datetime").alias("dropoff_datetime"), 
    col("passenger_count"),
    col("trip_distance"),
    col("RatecodeID"),
    col("store_and_fwd_flag"),
    col("PULocationID"),
    col("DOLocationID"),
    col("payment_type"),
    col("fare_amount"),
    col("extra"),
    col("mta_tax"),
    col("tip_amount"),
    col("tolls_amount"),
    col("ehail_fee"),
    col("improvement_surcharge"),
    col("total_amount"),
    col("trip_type"),
    col("congestion_surcharge"),
    lit(None).cast("double").alias("airport_fee"),
    lit("green").alias("taxi_type") 
)

print("Yellow taxi standardized columns:")
print(yellow_df_standardized.columns)
print(f"Count: {len(yellow_df_standardized.columns)}")

print("\nGreen taxi standardized columns:")
print(green_df_standardized.columns)
print(f"Count: {len(green_df_standardized.columns)}")

print(f"\nSchemas match: {yellow_df_standardized.columns == green_df_standardized.columns}")

print("4. Performing union...")
raw_trips_df = yellow_df_standardized.union(green_df_standardized)

print(f"\nCombined dataframe:")
print(f"- Total rows: {raw_trips_df.count():,}")
print(f"- Total columns: {len(raw_trips_df.columns)}")
print(f"- Yellow trips: {raw_trips_df.filter(col('taxi_type') == 'yellow').count():,}")
print(f"- Green trips: {raw_trips_df.filter(col('taxi_type') == 'green').count():,}")

print("\nSample of combined data:")
raw_trips_df.show(5, truncate=False)

print("\nFinal schema:")
raw_trips_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4. Performing union...
Yellow taxi standardized columns:
['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'trip_type', 'congestion_surcharge', 'airport_fee', 'taxi_type']
Count: 22

Green taxi standardized columns:
['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'trip_type', 'congestion_surcharge', 'airport_fee', 'taxi_type']
Count: 22

Schemas match: True
4. Performing union...

Combined dataframe:
- Total rows: 827,697,864
- Total columns: 22
- Yellow trips: 745,067,811
- Green trips: 82,630,053

Sample of c

In [57]:
print("5. Add calculate colume trip_duration_minutes")
raw_trips_with_duration = raw_trips_df.withColumn(
    "trip_duration_minutes", 
    (unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))) / 60
)
raw_trips_with_duration.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5. Add calculate colume trip_duration_minutes
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+---------+--------------------+-----------+---------+---------------------+
|VendorID|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|trip_type|congestion_surcharge|airport_fee|taxi_type|trip_duration_minutes|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+---------+--------------------+-----------+---------+-----------------

In [58]:
print("6. Filtering")

filtered_trips_df = raw_trips_with_duration.filter(
    (col("trip_distance") >= 0.1) &           
    (col("fare_amount") >= 2.0) &            
    (col("trip_duration_minutes") >= 1.0)  
)

print("After filtering:")

initial_count = raw_trips_with_duration.count()
final_count = filtered_trips_df.count()
removed_count = initial_count - final_count

print(f"Total trips: {final_count:,}")
print(f"Removed trips: {removed_count:,}")
print(f"Removal rate: {(removed_count/initial_count*100):.2f}%")

print("Filtered data statistics:")
print(f"Yellow trips: {filtered_trips_df.filter(col('taxi_type') == 'yellow').count():,}")
print(f"Green trips: {filtered_trips_df.filter(col('taxi_type') == 'green').count():,}")

print("Sample of filtered data:")
filtered_trips_df.select(
    "taxi_type", "pickup_datetime", "dropoff_datetime", 
    "trip_distance", "fare_amount", "trip_duration_minutes"
).show(5, truncate=False)
print(f"\nUpdated raw_trips_df with {raw_trips_df.count():,} valid trips")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6. Filtering
After filtering:
Total trips: 816,137,348
Removed trips: 11,560,516
Removal rate: 1.40%
Filtered data statistics:
Yellow trips: 735,821,800
Green trips: 80,315,548
Sample of filtered data:
+---------+-------------------+-------------------+-------------+-----------+---------------------+
|taxi_type|pickup_datetime    |dropoff_datetime   |trip_distance|fare_amount|trip_duration_minutes|
+---------+-------------------+-------------------+-------------+-----------+---------------------+
|yellow   |2014-01-01 00:29:18|2014-01-01 00:35:13|1.8          |7.5        |5.916666666666667    |
|yellow   |2014-01-01 00:36:33|2014-01-01 00:55:19|4.8          |17.5       |18.766666666666666   |
|yellow   |2014-01-01 00:20:43|2014-01-01 00:31:07|2.0          |9.5        |10.4                 |
|yellow   |2014-01-01 00:48:52|2014-01-01 00:56:14|1.1          |6.5        |7.366666666666666    |
|yellow   |2014-01-01 00:45:29|2014-01-01 00:55:33|1.6          |8.5        |10.066666666666666   

In [59]:
print("7. Adding time-based features...")

raw_trips_enhanced = filtered_trips_df.withColumn(
    "pickup_hour", hour(col("pickup_datetime"))
        ).withColumn(
    "pickup_day_of_week", dayofweek(col("pickup_datetime"))
        ).withColumn(
    "duration_min", 
    (unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))) / 60
        )

print("Time features added successfully!")
print("Sample with new time features:")
raw_trips_enhanced.select(
    "pickup_datetime", "pickup_hour", "pickup_day_of_week", "duration_min"
).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7. Adding time-based features...
Time features added successfully!
Sample with new time features:
+-------------------+-----------+------------------+------------------+
|    pickup_datetime|pickup_hour|pickup_day_of_week|      duration_min|
+-------------------+-----------+------------------+------------------+
|2014-01-01 00:29:18|          0|                 4| 5.916666666666667|
|2014-01-01 00:36:33|          0|                 4|18.766666666666666|
|2014-01-01 00:20:43|          0|                 4|              10.4|
|2014-01-01 00:48:52|          0|                 4| 7.366666666666666|
|2014-01-01 00:45:29|          0|                 4|10.066666666666666|
+-------------------+-----------+------------------+------------------+
only showing top 5 rows

In [60]:
print("8. Read taxi_zone_lookup.csv")
zone_lookup_schema = StructType([
    StructField("LocationID", LongType(), True),
    StructField("Borough", StringType(), True),
    StructField("Zone", StringType(), True),
    StructField("service_zone", StringType(), True),
])


zone_lookup_df = spark.read.csv(
    ZONE_LOOKUP_PATH, 
    header=True, 
    schema=zone_lookup_schema
)
print(f"Zone lookup data loaded: {zone_lookup_df.count()} zones")
print("Zone lookup schema:")
zone_lookup_df.printSchema()
print("\nSample zone data:")
zone_lookup_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8. Read taxi_zone_lookup.csv
Zone lookup data loaded: 265 zones
Zone lookup schema:
root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)


Sample zone data:
+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows

In [61]:
raw_trips_enhanced.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- trip_type: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- taxi_type: string (nullable = false)
 |-- trip_duration_mi

In [62]:
print("9. Join pickup_zones")
pickup_zones = zone_lookup_df.select(
    col("LocationID").alias("pickup_location_id"),
    col("Zone").alias("pickup_zone"),
    col("Borough").alias("pickup_borough"),
    col("service_zone").alias("pickup_service_zone")
)

trips_with_pickup_zones = raw_trips_enhanced.join(
    broadcast(pickup_zones),
    raw_trips_df.PULocationID == pickup_zones.pickup_location_id,
    "left"
).drop("pickup_location_id")

print("Show joined pickup_zones")
trips_with_pickup_zones.select("pickup_zone", "pickup_borough", "pickup_service_zone").show(5)
trips_with_pickup_zones.cache()
show_schema(trips_with_pickup_zones)

print("Pickup zones joined successfully!")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9. Join pickup_zones
Show joined pickup_zones
+--------------------+--------------+-------------------+
|         pickup_zone|pickup_borough|pickup_service_zone|
+--------------------+--------------+-------------------+
|Sutton Place/Turt...|     Manhattan|        Yellow Zone|
|     Lenox Hill East|     Manhattan|        Yellow Zone|
|            Elmhurst|        Queens|          Boro Zone|
|        Midtown East|     Manhattan|        Yellow Zone|
|            Flatiron|     Manhattan|        Yellow Zone|
+--------------------+--------------+-------------------+
only showing top 5 rows

StructField('VendorID', LongType(), True)
StructField('pickup_datetime', TimestampNTZType(), True)
StructField('dropoff_datetime', TimestampNTZType(), True)
StructField('passenger_count', LongType(), True)
StructField('trip_distance', DoubleType(), True)
StructField('RatecodeID', DoubleType(), True)
StructField('store_and_fwd_flag', StringType(), True)
StructField('PULocationID', LongType(), True)
Struct

In [63]:
print("10. Join dropoff_zones")
dropoff_zones = zone_lookup_df.select(
    col("LocationID").alias("dropoff_location_id"),
    col("Zone").alias("dropoff_zone"),
    col("Borough").alias("dropoff_borough"),
    col("service_zone").alias("dropoff_service_zone")
)

trips_with_all_zones = trips_with_pickup_zones.join(
    dropoff_zones,
    trips_with_pickup_zones.DOLocationID == dropoff_zones.dropoff_location_id,
    "left"
).drop("dropoff_location_id")

print("Show joined dropoff_zones")

trips_with_all_zones.printSchema()

print("Dropoff zones joined successfully!")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10. Join dropoff_zones
Show joined dropoff_zones
root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- trip_type: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- taxi_type

# TASK 3 Створення та агрегація фінального датафрейму zone_summary

In [75]:
print("Part 1: Creating zone_summary dataframe with basic aggregations...")

try:
    trips_with_all_zones.unpersist()
except:
    pass

trips_with_valid_zones = trips_with_all_zones.filter(col("pickup_zone").isNotNull())

print("Calculating basic metrics...")
zone_basic_metrics = trips_with_valid_zones.groupBy("pickup_zone").agg(
    count("*").alias("total_trips"),
    avg("trip_distance").alias("avg_trip_distance"),
    avg("total_amount").alias("avg_total_amount"),
    avg("tip_amount").alias("avg_tip_amount"),
    max("trip_distance").alias("max_trip_distance"),
    min("tip_amount").alias("min_tip_amount")
)
zone_basic_metrics.cache()
print(f"Basic metrics calculated for {zone_basic_metrics.count()} zones")

print("\nCalculating taxi type shares...")
taxi_type_counts = trips_with_valid_zones.groupBy("pickup_zone", "taxi_type") \
    .agg(count("*").alias("type_count"))

taxi_shares = taxi_type_counts.groupBy("pickup_zone").pivot("taxi_type", ["yellow", "green"]) \
    .sum("type_count") \
    .fillna(0)

taxi_shares_with_total = taxi_shares.join(
    zone_basic_metrics.select("pickup_zone", "total_trips"),
    "pickup_zone"
)

taxi_shares_final = taxi_shares_with_total.select(
    "pickup_zone",
    (col("yellow") / col("total_trips")).alias("yellow_share"),
    (col("green") / col("total_trips")).alias("green_share")
)

taxi_shares.unpersist()

print("Taxi shares calculated successfully")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Part 1: Creating zone_summary dataframe with basic aggregations...
Calculating basic metrics...
Basic metrics calculated for 262 zones

Calculating taxi type shares...
Taxi shares calculated successfully

In [76]:
print("Part 2: Joining all metrics...")

zone_summary = zone_basic_metrics.join(taxi_shares_final, "pickup_zone") \
    .select(
        "pickup_zone",
        "total_trips",
        "avg_trip_distance",
        "avg_total_amount",
        "avg_tip_amount",
        "yellow_share",
        "green_share",
        "max_trip_distance",
        "min_tip_amount"
    ) \
    .orderBy(col("total_trips").desc())

zone_basic_metrics.unpersist()

print("Zone summary schema:")
zone_summary.printSchema()

print("\nTop 10 zones by total trips:")
zone_summary.show(10, truncate=False)

print(f"\nTotal number of zones: {zone_summary.count()}")

print("\nVerifying yellow_share + green_share = 1:")
zone_summary.select(
    "pickup_zone",
    "yellow_share",
    "green_share",
    (col("yellow_share") + col("green_share")).alias("total_share")
).show(5)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Part 2: Joining all metrics...
Zone summary schema:
root
 |-- pickup_zone: string (nullable = true)
 |-- total_trips: long (nullable = false)
 |-- avg_trip_distance: double (nullable = true)
 |-- avg_total_amount: double (nullable = true)
 |-- avg_tip_amount: double (nullable = true)
 |-- yellow_share: double (nullable = true)
 |-- green_share: double (nullable = true)
 |-- max_trip_distance: double (nullable = true)
 |-- min_tip_amount: double (nullable = true)


Top 10 zones by total trips:
+----------------------------+-----------+------------------+------------------+------------------+------------------+---------------------+-----------------+--------------+
|pickup_zone                 |total_trips|avg_trip_distance |avg_total_amount  |avg_tip_amount    |yellow_share      |green_share          |max_trip_distance|min_tip_amount|
+----------------------------+-----------+------------------+------------------+------------------+------------------+---------------------+--------------

In [78]:
print("Part 3: Save results..)

output_path = f"s3://pzhoholiev-emr-bucket-hw/results/zone_statistic/{current_date}/zone_statistic.parquet"

print(f"\nSaving zone_summary to: {output_path}")
      
zone_summary.coalesce(1).write \
    .mode("overwrite") \
    .parquet(output_path)

print("Zone summary saved successfully!")
print("\nVerifying saved data...")
saved_zone_summary = spark.read.parquet(output_path)
print(f"Saved records count: {saved_zone_summary.count()}")
saved_zone_summary.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Saving zone_summary to: s3://pzhoholiev-emr-bucket-hw/results/zone_statistic/2025-07-19/zone_statistic.parquet
Zone summary saved successfully!

Verifying saved data...
Saved records count: 262
+--------------------+-----------+------------------+------------------+------------------+------------------+--------------------+-----------------+--------------+
|         pickup_zone|total_trips| avg_trip_distance|  avg_total_amount|    avg_tip_amount|      yellow_share|         green_share|max_trip_distance|min_tip_amount|
+--------------------+-----------+------------------+------------------+------------------+------------------+--------------------+-----------------+--------------+
|Upper East Side S...|   28757902| 4.958216156380258|12.717631308413935|1.3744157724023394|0.9999496486217945|5.035137820554503...|            1.2E7|         -0.84|
|      Midtown Center|   26879469| 6.282413865020874|15.451334278195722|1.7337466885972215| 0.999961234353253|3.876564674696512...|     1.1800000

# TASK 4 Агрегація по днях тижня та зонах

In [102]:
day_names = {
    1: "Monday",
    2: "Tuesday", 
    3: "Wednesday",
    4: "Thursday",
    5: "Friday",
    6: "Saturday",
    7: "Sunday"
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:
def get_trips_with_fares_over_30(week_day: Optional[int] = None):
    days_stats = []
    list_days_int_values = list(day_names.keys())
    if week_day is None:
        processed_days_list = list_days_int_values
    else:
        if week_day not in list_days_int_values:
            raise ValueError("Week_day arg can be between 1-7")
        processed_days_list = [week_day]
        
    for day in processed_days_list:
        trips_selected_day = trips_with_all_zones.filter(
            (col("pickup_day_of_week") == day) & 
            (col("pickup_zone").isNotNull())
        )
        
        zone_day_stats = trips_selected_day.groupBy("pickup_zone").agg(
            count("*").alias("trips_count"),

            sum(when(col("total_amount") > 30, 1).otherwise(0)).alias("high_fare_count"),

            avg("total_amount").alias("avg_total_amount"),
            avg("trip_distance").alias("avg_trip_distance"),

            max("total_amount").alias("max_fare"),
            min("total_amount").alias("min_fare")
        )
        
        zone_day_stats_final = zone_day_stats.withColumn(
            "high_fare_share",
            col("high_fare_count") / col("trips_count")
        ).withColumn(
            "day_of_week", 
            lit(day)
        ).withColumn(
            "day_name",
            lit(day_names[day])
        )
        
        zone_day_stats_final = zone_day_stats_final.select(
            "pickup_zone",
            "day_of_week",
            "day_name",
            "trips_count",
            "high_fare_share",
            "high_fare_count",
            "avg_total_amount",
            "avg_trip_distance",
            "max_fare",
            "min_fare"
        ).orderBy(col("trips_count").desc())
        days_stats.append(zone_day_stats_final)
    result = days_stats[0]
    if len(days_stats) > 1:
        for df in days_stats[1:]:
            result = result.union(df)
    return result
    
        
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
print("Stats on Monday")

monday_stats = get_trips_with_fares_over_30(1)
monday_stats.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Stats on Monday
+--------------------+-----------+--------+-----------+--------------------+---------------+------------------+------------------+---------+--------+
|         pickup_zone|day_of_week|day_name|trips_count|     high_fare_share|high_fare_count|  avg_total_amount| avg_trip_distance| max_fare|min_fare|
+--------------------+-----------+--------+-----------+--------------------+---------------+------------------+------------------+---------+--------+
|        East Village|          1|  Monday|    4191193| 0.05484691351603231|         229874|14.746358184888384|7.4885988881924686|  2759.37|     3.0|
|Penn Station/Madi...|          1|  Monday|    3400808| 0.06788210331191881|         230854|15.107021913613405|   7.4215919569702|  1026.66|     2.8|
|        Clinton East|          1|  Monday|    3341669| 0.07649889920276365|         255634|15.084945588563905|6.5619582998794845| 99619.02|     2.0|
|Times Sq/Theatre ...|          1|  Monday|    3130290| 0.11675371930396225|        

In [104]:
print("Stats all week days")

all_day_stats = get_trips_with_fares_over_30()
for day_num in day_names.keys():
    all_day_stats.filter(col("day_of_week") == day_num).show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Stats all week days
+----------------------------+-----------+--------+-----------+--------------------+---------------+------------------+------------------+---------+--------+
|pickup_zone                 |day_of_week|day_name|trips_count|high_fare_share     |high_fare_count|avg_total_amount  |avg_trip_distance |max_fare |min_fare|
+----------------------------+-----------+--------+-----------+--------------------+---------------+------------------+------------------+---------+--------+
|East Village                |1          |Monday  |4191193    |0.05484691351603231 |229874         |14.746358184888383|7.488598888192464 |2759.37  |3.0     |
|Penn Station/Madison Sq West|1          |Monday  |3400808    |0.06788210331191881 |230854         |15.10702191361341 |7.421591956970198 |1026.66  |2.8     |
|Clinton East                |1          |Monday  |3341669    |0.07649889920276365 |255634         |15.084945588563903|6.561958299879486 |99619.02 |2.0     |
|Times Sq/Theatre District   |1 

In [111]:
print("Save result task 4")
output_path = f"s3://pzhoholiev-emr-bucket-hw/results/zone_days_statistic/{current_date}/zone_days_statsti.parquet"

print(f"Saving zone day statistics to: {output_path}")
zone_day_stats_final.coalesce(1).write \
    .mode("overwrite") \
    .parquet(output_path)

print("Zone day statistics saved successfully!")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Save result task 4
Saving zone day statistics to: s3://pzhoholiev-emr-bucket-hw/results/zone_days_statistic/2025-07-19/zone_days_statsti.parquet
Zone day statistics saved successfully!